# MLIT API End-to-End Demo

This notebook exercises every currently documented MLIT Real Estate Information Library endpoint using the helper module in `mlit_api_demo.py`.

## Setup

- Ensure your `MLIT_API_KEY` is defined in the environment or `.env`.
- The helper exposes defaults for every endpoint so that the demo can run without extra input.

In [1]:
import pandas as pd
from IPython.display import display, Markdown

import mlit_api_demo as mlit

pd.set_option('display.max_columns', None)


def to_frame(payload):
    """Normalise assorted MLIT payloads into a DataFrame."""
    if isinstance(payload, dict):
        if 'data' in payload and isinstance(payload['data'], list):
            data = payload['data']
            if data and isinstance(data[0], dict):
                return pd.json_normalize(data)
            return pd.DataFrame({'value': data})
        if 'features' in payload and isinstance(payload['features'], list):
            rows = []
            for feature in payload['features']:
                props = feature.get('properties') or {}
                geom = feature.get('geometry') or {}
                row = dict(props)
                if geom.get('type'):
                    row['_geometry_type'] = geom['type']
                if 'coordinates' in geom:
                    row['_geometry_coordinates'] = geom['coordinates']
                rows.append(row)
            if rows:
                return pd.json_normalize(rows)
            return pd.DataFrame()
        return pd.json_normalize([payload])
    if isinstance(payload, list):
        if payload and isinstance(payload[0], dict):
            return pd.json_normalize(payload)
        return pd.DataFrame({'value': payload})
    return pd.DataFrame({'value': [payload]})

## Endpoint Catalogue

The helper exposes `ORDERED_SPECS` which carries human-readable metadata, required parameters, and sensible defaults for demos.

In [2]:
summary_rows = []
for spec in mlit.list_endpoints():
    summary_rows.append({
        'code': spec.code,
        'name': spec.name,
        'required': ', '.join(spec.required) if spec.required else 'None',
        'optional': ', '.join(spec.optional) if spec.optional else 'None',
        'defaults': mlit.get_default_params(spec.code)
    })
summary_df = pd.DataFrame(summary_rows)
display(summary_df)

,code,name,required,optional,defaults
0,XIT001,Real Estate Transaction Prices,year,"priceClassification, quarter, area, city, stat...","{'year': 2024, 'priceClassification': '01', 'a..."
1,XIT002,Prefecture Municipality Directory,area,language,"{'area': '04', 'language': 'en'}"
2,XCT001,Appraisal Report Information,"year, area, division",None,"{'year': 2024, 'area': '04', 'division': '00'}"
3,XPT001,Transaction Price Points,"response_format, z, x, y, from, to","priceClassification, landTypeCode","{'response_format': 'geojson', 'z': 13, 'x': 7..."
4,XPT002,Land Price Survey Points,"response_format, z, x, y, year","priceClassification, useCategoryCode","{'response_format': 'geojson', 'z': 13, 'x': 7..."
5,XKT001,City Planning GIS – Planning Areas,"response_format, z, x, y",None,"{'response_format': 'geojson', 'z': 13, 'x': 7..."
6,XKT002,City Planning GIS – Zoning,"response_format, z, x, y",None,"{'response_format': 'geojson', 'z': 13, 'x': 7..."
7,XKT003,City Planning GIS – Location Optimization Plan,"response_format, z, x, y",None,"{'response_format': 'geojson', 'z': 13, 'x': 7..."
8,XKT004,National Land Info – Elementary School Districts,"response_format, z, x, y",administrativeAreaCode,"{'response_format': 'geojson', 'z': 13, 'x': 7..."
9,XKT005,National Land Info – Junior High School Districts,"response_format, z, x, y",administrativeAreaCode,"{'response_format': 'geojson', 'z': 13, 'x': 7..."


## Sequential Demo Calls

Each endpoint is invoked with the published defaults. Adjust the `params` dictionary as you explore new slices.

In [3]:
results = []
for spec in mlit.list_endpoints():
    params = mlit.get_default_params(spec.code)
    print(f'{spec.code} - {spec.name}')
    print(f'Parameters: {params}')
    payload = mlit.call_endpoint(spec.code, **params)
    frame = to_frame(payload)
    results.append({
        'code': spec.code,
        'name': spec.name,
        'records': len(frame),
        'columns': list(frame.columns)[:6]
    })
    if not frame.empty:
        display(frame.head(3))
    else:
        print('No records returned.')
    print('-' * 80)

summary = pd.DataFrame(results)
display(summary)

XIT001 - Real Estate Transaction Prices
Parameters: {'year': 2024, 'priceClassification': '01', 'area': '04', 'quarter': 1, 'language': 'en'}


,PriceCategory,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,PricePerUnit,FloorPlan,Area,UnitPrice,LandShape,Frontage,TotalFloorArea,BuildingYear,Structure,Use,Purpose,Direction,Classification,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Renovation,Remarks
0,Real Estate Transaction Price Information,Residential Land(Land and Building),Residential Area,4101,Miyagi Prefecture,"Aoba Ward, Sendai City",Aobamachi,65000000,,,790,,Irregular Shaped,33,55,,W,House,House,Southeast,City Road,8,Category II Residential Zone,60,200,1st quarter 2024,,
1,Real Estate Transaction Price Information,Residential Land(Land and Building),Residential Area,4101,Miyagi Prefecture,"Aoba Ward, Sendai City",Asahigaoka,31000000,,,250,,Semi-rectangular Shaped,18,,,,,,South,Prefectural Road,66,Category I Residential Zone,60,200,1st quarter 2024,,
2,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,4101,Miyagi Prefecture,"Aoba Ward, Sendai City",Asahigaoka,32000000,,3LDK,75,,,,,2013,RC,House,House,,,,Category II Residential Zone,60,200,1st quarter 2024,Not yet,


--------------------------------------------------------------------------------
XIT002 - Prefecture Municipality Directory
Parameters: {'area': '04', 'language': 'en'}


,id,name
0,04100,Sendai City
1,04101,Aoba Ward
2,04102,Miyagino Ward


--------------------------------------------------------------------------------
XCT001 - Appraisal Report Information
Parameters: {'year': 2024, 'area': '04', 'division': '00'}


...

,_id,_index,plan_type_ja,group_code,prefecture,decision_type_ja,notice_number_s,plan_name,decision_date,city_name,notice_number,decision_maker,kubun_id,first_decision_date,_geometry_type,_geometry_coordinates
0,DRAt35gBJx74NFADZ_sT,bs027_district_plan_202508251201,地区計画,04100,宮城県,,,,,仙台市,,,1,,Polygon,"[[[140.8887255191803, 38.26477899682641], [140..."
1,VRAt35gBJx74NFADZ_oT,bs027_district_plan_202508251201,地区計画,04100,宮城県,,,,,仙台市,,,1,,Polygon,"[[[140.8887255191803, 38.264307261204806], [14..."
2,hRAt35gBJx74NFADZ_oT,bs027_district_plan_202508251201,地区計画,04100,宮城県,,,,,仙台市,,,1,,Polygon,"[[[140.8446729183197, 38.242848476219876], [14..."


--------------------------------------------------------------------------------
XKT024 - City Planning GIS – High-Use Districts
Parameters: {'response_format': 'geojson', 'z': 13, 'x': 7301, 'y': 3152}
No records returned.
--------------------------------------------------------------------------------
XKT025 - Liquefaction Propensity Map
Parameters: {'response_format': 'geojson', 'z': 13, 'x': 7301, 'y': 3152}


,_id,_index,topographic_classification_name_ja,mesh_code,note,topographic_classification_code,liquefaction_tendency_level,_geometry_type,_geometry_coordinates
0,qn_lt5IBJx74NFADmOLx,bs028_liquefaction_tendency_map_202410231432,砂礫質台地,5740362831,液状化しにくい,8,5,Polygon,"[[[140.8500051498413, 38.27273065049735], [140..."
1,nX_lt5IBJx74NFADmOLx,bs028_liquefaction_tendency_map_202410231432,砂礫質台地,5740362742,液状化しにくい,8,5,Polygon,"[[[140.8468723297119, 38.27273065049735], [140..."
2,rH_lt5IBJx74NFADmOLx,bs028_liquefaction_tendency_map_202410231432,砂礫質台地,5740362841,液状化しにくい,8,5,Polygon,"[[[140.85624933242798, 38.27273065049735], [14..."


--------------------------------------------------------------------------------


,code,name,records,columns
0,XIT001,Real Estate Transaction Prices,1468,"[PriceCategory, Type, Region, MunicipalityCode..."
1,XIT002,Prefecture Municipality Directory,40,"[id, name]"
2,XCT001,Appraisal Report Information,822,"[価格時点, 標準地番号 市区町村コード 県コード, 標準地番号 市区町村コード 市区町村コ..."
3,XPT001,Transaction Price Points,511,"[point_in_time_name_ja, price_information_cate..."
4,XPT002,Land Price Survey Points,86,"[_id, _index, location_number_ja, area_divisio..."
5,XKT001,City Planning GIS – Planning Areas,3,"[_id, _index, decision_date, city_name, prefec..."
6,XKT002,City Planning GIS – Zoning,54,"[_id, _index, prefecture, use_area_ja, city_co..."
7,XKT003,City Planning GIS – Location Optimization Plan,21,"[_id, _index, prefecture, city_code, notice_nu..."
8,XKT004,National Land Info – Elementary School Districts,16,"[_id, _index, A27_003, A27_004_ja, A27_001, A2..."
9,XKT005,National Land Info – Junior High School Districts,11,"[_id, _index, A32_004_ja, A32_001, A32_002, A3..."


## Next Steps

- Tweak the parameter dictionaries and rerun cells to explore new views.
- Persist interesting payloads by writing them to disk (e.g., `Path('data.json').write_text(...)`).
- The helper exposes function-level wrappers such as `fetch_transaction_prices` if you prefer direct imports.